# BAM Processing

> Processing BAM files for barcode extraction and classification

## Introduction
This notebook implements the BAM processing functionality for BarcodeSeqKit. It includes classes and functions for handling BAM files, extracting barcodes, and sorting reads into appropriate output files based on barcode detection.

In [50]:
#| default_exp bam_processing

In [51]:
#| hide
from nbdev.showdoc import *

In [85]:
#| export
import os
import re
import gzip
import logging
import shutil
from typing import List, Dict, Tuple, Set, Optional, Union, Iterator, Any, Counter
from pathlib import Path
import pysam
import tempfile
import pysam
import numpy as np
from pysam import AlignmentFile, AlignedSegment
import subprocess
from tqdm.auto import tqdm
from BarcodeSeqKit.core import (
    BarcodeConfig, 
    BarcodeMatch, 
    OrientationType,
    BarcodeExtractor, 
    ExtractionStatistics,
    BarcodeLocationType
)
#from BarcodeSeqKit.sequence_utils import classify_alignment_by_first_match
from BarcodeSeqKit.sequence_utils import classify_read_by_first_match

# BAM File Utilities
First, let's define utility functions for BAM file operations:

In [86]:
#| export
class BamUtils:
    """Utility functions for BAM file operations."""
    
    @staticmethod
    def sort_and_index(bam_file: str) -> None:
        """Sort and index a BAM file by reference.
        
        Args:
            bam_file: Path to the BAM file
        """
        # Create a temporary file for sorting
        temp_file = f"{bam_file}.temp.bam"
        
        # Sort the BAM file to a temporary file
        pysam.sort("-o", temp_file, bam_file)
        
        # Replace the original with the sorted file
        os.replace(temp_file, bam_file)
        
        # Index the BAM file
        pysam.index(bam_file)
    
    @staticmethod
    def get_read_count(bam_file: str) -> int:
        """Get the total number of reads in a BAM file.
        
        Args:
            bam_file: Path to the BAM file
            
        Returns:
            Total number of reads
        """
        # Run samtools idxstats to get read counts
        cmd = ["samtools", "idxstats", bam_file]
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        if result.returncode != 0:
            raise RuntimeError(f"Error running samtools idxstats: {result.stderr}")
        
        # Parse the output
        mapped = 0
        unmapped = 0
        for line in result.stdout.strip().split("\n"):
            if not line:
                continue
            parts = line.split("\t")
            if len(parts) >= 4:
                mapped += int(parts[2])
                unmapped += int(parts[3])
        
        return mapped + unmapped
    
    @staticmethod
    def merge_bam_files(output_file: str, input_files: List[str]) -> None:
        """Merge multiple BAM files into one.
        
        Args:
            output_file: Path to the output BAM file
            input_files: List of input BAM file paths
        """
        if not input_files:
            return
        
        # Use pysam to merge the files
        pysam.merge("-f", output_file, *input_files)

# BAM Extractor
Now let's implement the main BAM extractor class that inherits from the BarcodeExtractor base class:

In [92]:
#| export
class BamExtractor(BarcodeExtractor):
    """Extracts barcodes from BAM files."""
    
    def __init__(
        self, 
        barcodes: List[BarcodeConfig],
        output_prefix: str,
        bam_file: str,
        output_dir: Optional[str] = None,
        merge_orientations: bool = False,
        keep_unmatched: bool = True,
        verbose: bool = False,
        log_file: Optional[str] = None,
        max_mismatches: int = 0,
        search_mate: bool = True
    ):
        """Initialize the BAM extractor.
        
        Args:
            barcodes: List of barcode configurations
            output_prefix: Prefix for output files
            bam_file: Path to the input BAM file
            output_dir: Directory for output files
            merge_orientations: Whether to merge forward and reverse complement orientations
            keep_unmatched: Whether to keep reads without barcodes
            verbose: Whether to enable verbose logging
            log_file: Path to log file
            max_mismatches: Maximum number of mismatches allowed in barcode detection
            search_mate: Whether to search for barcodes in mate reads
        """
        super().__init__(
            barcodes=barcodes,
            output_prefix=output_prefix,
            output_dir=output_dir,
            merge_orientations=merge_orientations,
            keep_unmatched=keep_unmatched,
            verbose=verbose,
            log_file=log_file
        )
        
        # Validate BAM file
        if not os.path.exists(bam_file):
            raise FileNotFoundError(f"BAM file not found: {bam_file}")
        
        self.bam_file = bam_file
        self.max_mismatches = max_mismatches
        self.search_mate = search_mate
        
        # Check if BAM file is sorted and indexed
        try:
            # Try to open the index file
            pysam.AlignmentFile(bam_file, "rb").check_index()
            self.is_indexed = True
        except (ValueError, IOError):
            self.logger.warning(f"BAM file is not indexed: {bam_file}")
            self.logger.info("Sorting and indexing BAM file...")
            BamUtils.sort_and_index(bam_file)
            self.is_indexed = True
        
        # Prepare output categories based on barcodes
        self.categories = self._prepare_categories()
        
        # Count the total number of reads
        self.total_reads = BamUtils.get_read_count(bam_file)
        
        self.logger.info(f"Initialized BamExtractor with {len(barcodes)} barcodes")
        self.logger.info(f"BAM file: {bam_file} ({self.total_reads} reads)")
        self.logger.info(f"Output categories: {self.categories}")
    
    def _prepare_categories(self) -> List[str]:
        """Prepare output categories based on barcodes.
        
        Returns:
            List of category names
        """
        categories = []
        
        # Single barcode mode (no location specified) or multiple barcodes
        single_barcode_mode = all(b.location.value == "UNK" for b in self.barcodes)
        
        if single_barcode_mode:
            if self.merge_orientations:
                categories.append("barcode_orientAll")
            else:
                categories.extend(["barcode_orientFR", "barcode_orientRC"])
        else:
            # Multiple barcodes with locations
            for barcode in self.barcodes:
                location = barcode.location.value
                if location in ["5", "3"]:
                    if self.merge_orientations:
                        categories.append(f"barcode{location}_orientAll")
                    else:
                        categories.extend([
                            f"barcode{location}_orient{OrientationType.FORWARD.value}",
                            f"barcode{location}_orient{OrientationType.REVERSE_COMPLEMENT.value}"
                        ])
        
        # Add combined categories
        if len(self.barcodes) > 1 and not single_barcode_mode:
            categories.append("combined_barcodeAll")
        
        # Add no barcode category
        if self.keep_unmatched:
            categories.append("noBarcode")
        
        return categories
    
    def extract(self) -> ExtractionStatistics:
        """Extract barcodes from the BAM file.
        
        Returns:
            Statistics from the extraction process
        """
        self.logger.info("Starting barcode extraction from BAM file")
        
        # Initialize statistics
        stats = ExtractionStatistics()
        stats.total_reads = self.total_reads
        
        # Open the input BAM file
        bamfile = pysam.AlignmentFile(self.bam_file, "rb")
        
        # Initialize output BAM files
        output_files = self._init_output_files(bamfile)
        
        # Track processed reads to avoid duplicates
        #processed_reads = set()
        
        # First pass: classify reads by barcode
        #store only of there is a match
        read_classifications = {}
        
        # Process each read in the BAM file
        read_count = 0
        
        try:
            for read in bamfile:
                read_count += 1
                
                # Skip if we've already processed this read or its mate
                if read.query_name in read_classifications:
                    continue
                               
                # Get the read sequence
                sequence = read.query_sequence
                
                if not sequence:
                    continue
                
                # Search for barcodes
                match, category = classify_read_by_first_match(
                    sequence=sequence,
                    barcodes=self.barcodes,
                    max_mismatches=self.max_mismatches
                )
                
                # Update statistics
                if match:
                    stats.update_barcode_match(match)
                    read_classifications[read.query_name] = category
                else:
                    stats.no_barcode_count += 1
                
            
            self.logger.info(f"First pass complete: classified {len(read_classifications)} reads")
            bamfile.close()
            # Reset for second pass
            bamfile = pysam.AlignmentFile(self.bam_file, "rb")
            read_count = 0
            
            # Second pass: write reads to output files
            # if not in read_classifications = noBarcode
            for read in bamfile:
                read_count += 1
                
                category = read_classifications.get(read.query_name, "noBarcode")
                
                #Write to output file
                if category != "noBarcode":
                    #print('im writing', read, 'to', output_files[category])
                    output_files[category].write(read)
                
        finally:
            # Close all file handles
            bamfile.close()
            for k,f in  zip(output_files.keys(), output_files.values()):
                print('output_files',k,f)
                f.close()
        
        #print(read_classifications)
        self.logger.info(f"Extraction complete: {stats.total_barcode_matches} matches in {stats.total_reads} reads")
        
        # Sort and index all output files
        for category in output_files:
            file_path = self._get_output_path(category)
            self.logger.info(f"Sorting and indexing {file_path}")
            BamUtils.sort_and_index(file_path)
        
        # Save statistics
        self.save_statistics()
        
        # Create merged files if requested
        if self.merge_orientations:
            self._create_merged_files()
        
        return stats
    
    def _init_output_files(self, template_file: pysam.AlignmentFile) -> Dict[str, pysam.AlignmentFile]:
        """Initialize output BAM files.
        
        Args:
            template_file: Template BAM file
            
        Returns:
            Dictionary mapping categories to output file handles
        """
        output_files = {}
        
        for category in self.categories:
            output_path = self._get_output_path(category)
            output_files[category] = pysam.AlignmentFile(
                output_path, "wb", template=template_file
            )
        
        return output_files
    
    def _get_output_path(self, category: str) -> str:
        """Get the output path for a category.
        
        Args:
            category: Category name
            
        Returns:
            Path to the output BAM file
        """
        filename = f"{self.output_prefix}_{category}.bam"
        return os.path.join(self.output_dir, filename)
    
    def _create_merged_files(self):
        """Create merged files for combined categories."""
        self.logger.info("Creating merged files for combined categories")
        
        # Check if we need to create a combined file for all barcodes
        single_barcode_mode = all(b.location.value == "UNK" for b in self.barcodes)
        barcode_locations = set(b.location.value for b in self.barcodes if b.location.value in ["5", "3"])
        
        # Merge orientation files for each barcode location
        for location in barcode_locations:
            out_file = os.path.join(
                self.output_dir, 
                f"{self.output_prefix}_combined_barcode{location}.bam"
            )
            
            # Find all files for this barcode location
            pattern = f"{self.output_prefix}_barcode{location}_orient*.bam"
            
            input_files = []
            for f in os.listdir(self.output_dir):
                if re.match(pattern, f):
                    input_files.append(os.path.join(self.output_dir, f))
            
            if input_files:
                self.logger.info(f"Merging {len(input_files)} files into {out_file}")
                BamUtils.merge_bam_files(out_file, input_files)
                BamUtils.sort_and_index(out_file)
        
        # Create a combined file for all barcodes if needed
        if len(barcode_locations) > 1:
            out_file = os.path.join(
                self.output_dir, 
                f"{self.output_prefix}_combined_barcodeAll.bam"
            )
            
            # Find all barcode files
            pattern = f"{self.output_prefix}_barcode[53]_orient*.bam"
            
            input_files = []
            for f in os.listdir(self.output_dir):
                if re.match(pattern, f):
                    input_files.append(os.path.join(self.output_dir, f))
            
            if input_files:
                self.logger.info(f"Merging {len(input_files)} files into {out_file}")
                BamUtils.merge_bam_files(out_file, input_files)
                BamUtils.sort_and_index(out_file)
    
    def _find_barcode_matches(self, sequence: str) -> List[BarcodeMatch]:
        """Find barcode matches in a sequence.
        
        Note: This implementation uses classify_read_by_first_match instead of
        finding all matches for efficiency.
        
        Args:
            sequence: The sequence to search in
            
        Returns:
            List of BarcodeMatch objects
        """
        match, _ = classify_read_by_first_match(
            sequence=sequence,
            barcodes=self.barcodes,
            max_mismatches=self.max_mismatches
        )
        
        if match:
            return [match]
        else:
            return [] 

# Conclusion
This notebook implements the BAM processing functionality for BarcodeSeqKit, providing efficient extraction and classification of barcoded reads from BAM files. The implementation handles sorting, indexing, and merging of BAM files, and provides a flexible way to classify reads based on barcode presence and orientation.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()